In [7]:
import sqlparse
import os
import sys

sys.path.append(f'z:\OMOP\omop_etl')

import pandas as pd
# os.chdir('z:/Covid19/Covid19_OMOP/new_pipeline')
# print(os.getcwd())

from omop_etl.datastore import DataStore
from omop_etl.load import Loader
from omop_etl.utils import search
from omop_etl.bo import bo_query, format_stage_query

omop = DataStore('Z:/omop_proj_test/config.yml')
loader = Loader('Z:/omop_proj_test/config.yml')

# Configuration files

In [3]:
from omop_etl.config import ProjectConfig, ETLConfig

# BO Queries

In [8]:
omop = DataStore('config.yml')

# def format_stage_query(dp_name):

#     with open('omop_etl/etl_config.yml') as f:
#         yml = yaml.safe_load(f) 
    
#     aliases= yml['aliases'][dp_name]
#     bo_queries = omop.get_bo_query('omop')

#     start_date = omop.config_param['date_range']['start_date']
#     end_date = omop.config_param['date_range']['end_date']

#     sql_query = format_bo_sql(bo_queries[dp_name], dp_name, schema='stage', aliases=aliases)
#     patient_id = "select PATIENT_KEY from DWS_OMOP.cohort.PersonList"
#     sql_query = sql_query.replace("12345678", patient_id)
#     sql_query = sql_query.replace("01/01/1900 00:0:0", start_date)
#     sql_query = sql_query.replace("12/31/1900 00:0:0", end_date)

#     sql_query = sqlparse.format(sql_query, reindent_aligned=True, indent_with=1)

#     return f"EXECUTE ('USE [DWS_PROD];\n {sql_query}')" 

## BO cohort queries

In [5]:
from omop_etl.bo import bo_query
with omop.bo_engine.connect() as con:
    bo_queries = bo_query('omop', con)

len(bo_queries.keys())
# [q for q in bo_queries.keys() if bo_queries[q] == '']

64

In [8]:
# bo_queries = omop..get_bo_query('omop')
sqlstring = bo_queries['MEASUREMENT_BP_CVP']#'MEASUREMENT_BP_MAP_Cuff'] or 'MEASUREMENT_Height' MEASUREMENT_Weight
parsed = sqlparse.parse(sqlstring)[0]
idx = [parsed.token_index(t) for t in parsed if t.is_keyword and t.value == 'FROM'][0]
columns = parsed.token_prev(idx)[1]

# Extract columns from SELECT clause. If duplicated columns, use alias, 
# else append abbreviated source table name.
colnames = [i.value.split('.')[-1] for i in columns]
dup_cols = set([x for x in colnames if colnames.count(x) > 1])
new_colnames = []

counter = 0
for item in columns:
    if isinstance(item, sqlparse.sql.Identifier):
        colname = item.value.split('.')[-1]
        tabname = item.value.split('.')[-2]
        shrt_tabname = '_'.join([word[:3] for word in tabname.split('_')])
        # print('Identifier', item.value)
    
    elif isinstance(item, sqlparse.sql.Function):
        counter += 1
        fn_name = f'FN_{counter}'
        for x in item:
            if (isinstance(x, sqlparse.sql.Identifier) and x.value == 'cast'):
                print(item.value)
                continue
            else: 
                pass
            if isinstance(x, sqlparse.sql.Parenthesis): 
                for y in x:
                    if isinstance(y, sqlparse.sql.Identifier):
                        print(y.value)
                        print([i.strip() for i in y.value.split(' as ')])
                        # col, dtype = [i.strip() for i in y.value.split('as')]
                        # print(f'try_convert({dtype}, {col})')
    
    # else: 
    #     print('Something else', item.__repr__)

cast(Flowsheet_BP_CVP.MEASR_VALUE as numeric(10,2))
Flowsheet_BP_CVP.MEASR_VALUE as numeric(10,2)
['Flowsheet_BP_CVP.MEASR_VALUE', 'numeric(10,2)']


In [55]:
loj_idxs = [parsed.token_index(t) for t in parsed if t.is_keyword and t.value == 'LEFT OUTER JOIN']

for idx in loj_idxs:
    print(idx, parsed.token_prev(idx))


30 (27, <Parenthesis '(ALL_P...' at 0x1FA66F0FD48>)
48 (45, <Parenthesis '(ALL_P...' at 0x1FA66F0FF48>)
57 (54, <Parenthesis '(dbo.P...' at 0x1FA66F0FE48>)
66 (63, <Parenthesis '(dbo.P...' at 0x1FA66FD9148>)
75 (72, <Parenthesis '(Flows...' at 0x1FA66FD9348>)
84 (81, <Parenthesis '(Flows...' at 0x1FA66FD9448>)


In [61]:
tok = parsed.token_prev(48)[1]
tok.value

'(ALL_PROVIDER_IDS_ATTEND_SER.IDENT_ID_TYPE= -1001\r  AND ALL_PROVIDERS_ATTENDING.PROVIDR_KEY=ALL_PROVIDER_IDS_ATTEND_SER.PROVIDR_KEY AND ALL_PROVIDER_IDS_ATTEND_SER.IDENT_ID_TYPE= -1001)'

In [99]:
import numpy as np

# bo_queries = omop.get_bo_query('omop')
sqlstring = bo_queries['MEASUREMENT_Height']#'MEASUREMENT_BP_MAP_Cuff'] or 'MEASUREMENT_Height' MEASUREMENT_Weight MEASUREMENT_BP_CVP
parsed = sqlparse.parse(sqlstring)[0]

def get_function(item):
    if isinstance(item, (sqlparse.sql.Parenthesis, sqlparse.sql.Operation)):
        return list(filter(None, [get_function(a) for a in item]))
    elif isinstance(item, sqlparse.sql.Function):
        return item

def flatten(lst):
    for el in lst:
        if isinstance(el, list):  
            # recurse
            yield from flatten(el)
        else:
            # generate
            yield el
 

# def remove_distinct_nulls_filter(parsed):
#     """Remove distinct clause and nulls filter from subquery. 

#     This modification was introduced to improve BO queries performance.
#     """
for token in parsed.tokens:
    if isinstance(token, sqlparse.sql.Identifier):# (sqlparse.sql.IdentifierList, sqlparse.sql.Where)):
        items = [item for item in token if search('SELECT', item.value)]
        if items:
            for item in items:
                item.value = item.value\
                                 .lower()\
                                 .replace('distinct a.link_patnt_encntr_key',' a.link_patnt_encntr_key')\
                                 .replace('and a.measr_value is not null', '')

# remove_distinct_nulls_filter(parsed)
        # for item in items:
        #     print(item.value)
            # if isinstance(item, sqlparse.sql.Function):
            #     fun_list = flatten([get_function(item)])
                # print(fun_list)

        #     else:
        #         fun_list = flatten(list(filter(None, [get_function(i) for i in item])))
        #         # print(fun_list)

        #     for fun in fun_list:
        #         col, dtype = flatten([[i.value.split(' as ') for i in p if isinstance(i, sqlparse.sql.Identifier)] 
        #                                 for p in fun if isinstance(p, sqlparse.sql.Parenthesis)][0])
        #         # print(fun)
        #         item.value = item.value.replace(fun.value, f'try_convert({dtype},{col})')
    
        # token.value = ''.join(item.value for item in token)

        # print(token.value)

In [100]:
_list = []
for token in parsed.tokens:
    if isinstance(token, sqlparse.sql.Identifier):# (sqlparse.sql.IdentifierList, sqlparse.sql.Where)):
        items = [item for item in token if search('SELECT', item.value)]
        if items:
            _list.append(items)

In [101]:
_list

[]

In [ ]:
from functools import reduce

In [ ]:
reduce(list.__add__, (list(i) for i in [[i.value.split(' as ') for i in p if isinstance(i, sqlparse.sql.Identifier)] 
                                        for p in fun if isinstance(p, sqlparse.sql.Parenthesis)][0]))

In [ ]:
fun_list

In [ ]:
for token in parsed.tokens:
#     print(token.ttype)
    if isinstance(token, sqlparse.sql.IdentifierList) or isinstance(token, sqlparse.sql.Where):
        print(token.ttype)

In [ ]:
import sqlparse

def extract_definitions(token_list):
    # assumes that token_list is a parenthesis
    definitions = []
    tmp = []
    par_level = 0
    for token in token_list.flatten():
        if token.is_whitespace:
            continue
        elif token.match(sqlparse.tokens.Punctuation, '('):
            par_level += 1
            continue
        if token.match(sqlparse.tokens.Punctuation, ')'):
            if par_level == 0:
                break
            else:
                par_level += 1
        elif token.match(sqlparse.tokens.Punctuation, ','):
            if tmp:
                definitions.append(tmp)
            tmp = []
        else:
            tmp.append(token)
    if tmp:
        definitions.append(tmp)
    return definitions